## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page: [ICD-O Clinical Terminology Mapper](https://aws.amazon.com/marketplace/pp/prodview-n65m4efre6bpg)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## ICD-O Clinical Terminology Mapper

- **Model**: `icdo_vdb_resolver`
- **Model Description**: This pretrained pipeline extracts oncological entities from clinical text and maps them to their corresponding ICD-O codes.

In [24]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [25]:
import json
import os
import boto3
import pandas as pd
import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from IPython.display import display
from urllib.parse import urlparse


In [26]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [27]:
model_name = "icdo-vdb-resolver"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [28]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

Once endpoint has been created, you would be able to perform real-time inference.

In [30]:
def invoke_realtime_endpoint(record, content_type="application/json", accept="application/json"):
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=json.dumps(record) if content_type == "application/json" else record,
    )

    response_body = response["Body"].read().decode("utf-8")

    if accept == "application/json":
        return json.loads(response_body)
    elif accept == "application/jsonlines":
        return response_body
    else:
        raise ValueError(f"Unsupported accept type: {accept}")

### Initial Setup

In [31]:
docs = [
    "A few studies have demonstrated that ANGPTL1 functions as a tumor suppressor gene in breast cancer  , hepatocellular carcinoma  , colorectal cancer and parathyroid carcinoma.", 
    "TRAF6 is a putative oncogene in a variety of cancers including  bladder cancer , and skin cancer. WWP2 appears to regulate the expression of the well characterized tumor suppressor phosphatase and tensin homolog (PTEN) in endometrial cancer and squamous cell carcinoma.",
]

sample_text = "TRIM50 has only been shown to act as a tumor suppressor in hepatocellular carcinoma and ovary cancer. HOXA10 exerts an oncogenic role in several tumors endometrial adenocarcinoma ."

### JSON

In [32]:
input_json_data = {"text": sample_text}
response_json = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")
pd.DataFrame(response_json["predictions"][0])

,begin,end,ner_chunk,ner_label,ner_confidence,concept_code,resolution,score,all_codes,all_resolutions,all_score
0,39,54,tumor suppressor,Oncological,0.76909995,8001/1,tumor cells,0.719942,"[8001/1, 8000/1, 8000/3, 8001/0, 8040/1]","[tumor cells, tumor, cancer, tumor cells, benign, tumorlet]","[0.7199416756629944, 0.7053937911987305, 0.6986449956893921, 0.6981863379478455, 0.6867451667785645]"
1,59,82,hepatocellular carcinoma,Cancer_dx,None,8170/3,hepatocellular carcinoma,1.000000,"[8170/3, 8170/3, 8170/3, 8170/3-C22.0, 8170/3]","[hepatocellular carcinoma, hepatocarcinoma, liver cell carcinoma, hepatocellular carcinoma, of liver, hepatoma]","[1.0000004768371582, 0.9132293462753296, 0.9028671979904175, 0.8793962597846985, 0.8304914832115173]"
2,88,99,ovary cancer,Cancer_dx,None,8010/3-C56.9,"carcinoma, of ovary",0.840850,"[8010/3-C56.9, 8140/3-C56.9, 8010/6-C56.9, 8000/3-C56.9, 8230/3-C56.9]","[carcinoma, of ovary, adenocarcinoma, of ovary, carcinoma, metastatic, of ovary, neoplasm, malignant of ovary, solid carcinoma, of ovary]","[0.8408499360084534, 0.8210016489028931, 0.8171762228012085, 0.8160241842269897, 0.8076353669166565]"
3,145,177,tumors endometrial adenocarcinoma,Oncological,0.8207,8140/3-C54.1,"adenocarcinoma, of endometrium",0.897595,"[8140/3-C54.1, 8380/3, 8380/3-C54.1, 8380/3-C55.9, 8260/3-C54.1]","[adenocarcinoma, of endometrium, endometrioid adenocarcinoma, endometrioid adenocarcinoma, of endometrium, endometrioid adenocarcinoma, of uterus, papillary adenocarcinoma, of endometrium]","[0.8975948095321655, 0.8903557062149048, 0.8701358437538147, 0.8674242496490479, 0.8599061965942383]"


### JSON Lines

In [34]:
def create_jsonl(records):
    if isinstance(records, str):
        records = [records]
    json_records = [{"text": text} for text in records]
    json_lines = "\n".join(json.dumps(record) for record in json_records)
    return json_lines

In [35]:
input_jsonl_data = create_jsonl(sample_text)
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"begin": 39, "end": 54, "ner_chunk": "tumor suppressor", "ner_label": "Oncological", "ner_confidence": "0.76909995", "concept_code": "8001/1", "resolution": "tumor cells", "score": 0.7199416756629944, "all_codes": ["8001/1", "8000/1", "8000/3", "8001/0", "8040/1"], "all_resolutions": ["tumor cells", "tumor", "cancer", "tumor cells, benign", "tumorlet"], "all_score": [0.7199416756629944, 0.7053937911987305, 0.6986449956893921, 0.6981863379478455, 0.6867451667785645]}, {"begin": 59, "end": 82, "ner_chunk": "hepatocellular carcinoma", "ner_label": "Cancer_dx", "ner_confidence": null, "concept_code": "8170/3", "resolution": "hepatocellular carcinoma", "score": 1.0000004768371582, "all_codes": ["8170/3", "8170/3", "8170/3", "8170/3-C22.0", "8170/3"], "all_resolutions": ["hepatocellular carcinoma", "hepatocarcinoma", "liver cell carcinoma", "hepatocellular carcinoma, of liver", "hepatoma"], "all_score": [1.0000004768371582, 0.9132293462753296, 0.9028671979904175, 0.87939625

### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [38]:
validation_json_file_name = "input.json"
validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/jsonl/"

def upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_format[1:]}/{file_name}",
        Body=input_data.encode("UTF-8"),
    )

In [39]:
# Create JSON and JSON Lines data
input_jsonl_data = create_jsonl(docs)
input_json_data = json.dumps({"text": docs})

# Upload JSON and JSON Lines data to S3
upload_to_s3(input_json_data, validation_json_file_name)
upload_to_s3(input_jsonl_data, validation_jsonl_file_name)

### JSON

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [ ]:
def retrieve_json_output_from_s3(validation_file_name):
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

In [43]:
retrieve_json_output_from_s3(validation_json_file_name)

{'predictions': [[{'begin': 60,
    'end': 75,
    'ner_chunk': 'tumor suppressor',
    'ner_label': 'Oncological',
    'ner_confidence': '0.6416',
    'concept_code': '8001/1',
    'resolution': 'tumor cells',
    'score': 0.7199416160583496,
    'all_codes': ['8001/1', '8000/1', '8000/3', '8001/0', '8040/1'],
    'all_resolutions': ['tumor cells',
     'tumor',
     'cancer',
     'tumor cells, benign',
     'tumorlet'],
    'all_score': [0.7199416160583496,
     0.7053937911987305,
     0.6986448764801025,
     0.6981862783432007,
     0.6867449879646301]},
   {'begin': 85,
    'end': 97,
    'ner_chunk': 'breast cancer',
    'ner_label': 'Cancer_dx',
    'ner_confidence': None,
    'concept_code': 'C50',
    'resolution': 'breast',
    'score': 0.83452969789505,
    'all_codes': ['C50',
     '8010/3-C50.9',
     '8000/3-C50.9',
     '8010/6-C50.9',
     '8000/6-C50.9'],
    'all_resolutions': ['breast',
     'carcinoma, of breast',
     'neoplasm, malignant of breast',
     'carcin

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [ ]:
def retrieve_jsonlines_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

In [46]:
retrieve_jsonlines_output_from_s3(validation_jsonl_file_name)

{"predictions": [{"begin": 60, "end": 75, "ner_chunk": "tumor suppressor", "ner_label": "Oncological", "ner_confidence": "0.6416", "concept_code": "8001/1", "resolution": "tumor cells", "score": 0.7199416160583496, "all_codes": ["8001/1", "8000/1", "8000/3", "8001/0", "8040/1"], "all_resolutions": ["tumor cells", "tumor", "cancer", "tumor cells, benign", "tumorlet"], "all_score": [0.7199416160583496, 0.7053937911987305, 0.6986448764801025, 0.6981862783432007, 0.6867449879646301]}, {"begin": 85, "end": 97, "ner_chunk": "breast cancer", "ner_label": "Cancer_dx", "ner_confidence": null, "concept_code": "C50", "resolution": "breast", "score": 0.83452969789505, "all_codes": ["C50", "8010/3-C50.9", "8000/3-C50.9", "8010/6-C50.9", "8000/6-C50.9"], "all_resolutions": ["breast", "carcinoma, of breast", "neoplasm, malignant of breast", "carcinoma, metastatic, of breast", "neoplasm, metastatic of breast"], "all_score": [0.83452969789505, 0.8012375831604004, 0.7926520109176636, 0.7858340740203857,

In [47]:
model.delete_model()

INFO:sagemaker:Deleting model with name: icdo-vdb-resolver-2025-08-15-14-41-12-751


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

